# Generate dollar bars
courtesy of towards data science

In [19]:
import pandas as pd
import numpy as np
import datetime
from IPython.display import clear_output

In [2]:
df = pd.read_csv("../data/BTCUSDT-trades-100.csv")

In [3]:
df

,a,p,q,f,l,T,m,M
0,500000000,40101.08,0.105000,556555238,556555238,1610164456730,True,True
1,500000001,40101.07,0.004500,556555239,556555239,1610164456730,True,True
2,500000002,40101.06,0.003567,556555240,556555240,1610164456730,True,True
3,500000003,40099.92,0.014772,556555241,556555241,1610164456784,True,True
4,500000004,40100.71,0.002105,556555242,556555242,1610164456821,False,True
...,...,...,...,...,...,...,...,...
4999995,504999995,33274.26,0.021294,562215962,562215962,1610349968281,False,True
4999996,504999996,33270.62,0.003558,562215963,562215963,1610349968286,False,True
4999997,504999997,33268.82,0.012837,562215964,562215964,1610349968287,False,True
4999998,504999998,33270.62,0.001745,562215965,562215965,1610349968287,False,True


In [4]:
df_np = df[["T", "p", "q"]].to_numpy()

In [38]:
# expects a numpy array with trades
# each trade is composed of: [time, price, quantity]
def generate_dollarbars(trades, frequency=100000):
    times = trades[:,0]
    prices = trades[:,1]
    volumes = trades[:,2]
    ans = np.zeros(shape=(len(prices), 6))
    candle_counter = 0
    dollars = 0
    lasti = 0
    carry = 0
    i = 0
    while i < prices.shape[0]:
        dollars += volumes[i]*prices[i]
        if dollars >= frequency:
            carry = (dollars-frequency)/prices[i]
            times = np.insert(times, i+1, times[i])
            prices = np.insert(prices, i+1, prices[i])
            volumes = np.insert(volumes, i+1, carry)
            ans[candle_counter][0] = times[i]                          # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])-carry  # volume
            candle_counter += 1
            lasti = i+1
            dollars = 0
            clear_output()
            print(f"currently on index {i} out of {prices.shape[0]}")
        i += 1
    return ans[:candle_counter]

In [5]:
# Towards Data Science base
# expects a numpy array with trades
# each trade is composed of: [time, price, quantity]
def generate_dollarbars(trades, frequency=100000):
    times = trades[:,0]
    prices = trades[:,1]
    volumes = trades[:,2]
    ans = np.zeros(shape=(len(prices), 6))
    candle_counter = 0
    dollars = 0
    lasti = 0
    for i in range(len(prices)):
        dollars += volumes[i]*prices[i]
        if dollars >= frequency:
            ans[candle_counter][0] = times[i]                          # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])        # volume
            candle_counter += 1
            lasti = i+1
            dollars = 0
#             clear_output()
#             print(f"currently on index {i} out of {prices.shape[0]}")
    return ans[:candle_counter]

In [6]:
a = generate_dollarbars(df_np)

In [7]:
a

array([[1.61016446e+12, 4.01010800e+04, 4.01010800e+04, 4.00953900e+04,
        4.00953900e+04, 2.54893600e+00],
       [1.61016447e+12, 4.00953800e+04, 4.01190100e+04, 4.00953600e+04,
        4.01190100e+04, 2.58144700e+00],
       [1.61016448e+12, 4.01153000e+04, 4.01295200e+04, 4.01153000e+04,
        4.01294400e+04, 2.65717100e+00],
       ...,
       [1.61034997e+12, 3.32873300e+04, 3.32873400e+04, 3.32386600e+04,
        3.32386600e+04, 3.07135300e+00],
       [1.61034997e+12, 3.32370600e+04, 3.33017600e+04, 3.32271900e+04,
        3.33008300e+04, 3.10929300e+00],
       [1.61034997e+12, 3.32843800e+04, 3.33009700e+04, 3.32742300e+04,
        3.32742300e+04, 3.01710000e+00]])

In [15]:
df = pd.DataFrame(a, columns=["time", "open", "high", "low", "close", "volume"])

In [16]:
df

,time,open,high,low,close,volume
0,1.610164e+12,40101.08,40101.08,40095.39,40095.39,2.548936
1,1.610164e+12,40095.38,40119.01,40095.36,40119.01,2.581447
2,1.610164e+12,40115.30,40129.52,40115.30,40129.44,2.657171
3,1.610164e+12,40129.44,40129.44,40129.35,40129.35,2.988235
4,1.610164e+12,40129.35,40129.35,40100.00,40102.96,3.588482
...,...,...,...,...,...,...
84251,1.610350e+12,33283.27,33312.49,33277.92,33291.65,3.225255
84252,1.610350e+12,33291.65,33306.28,33272.62,33287.33,3.040252
84253,1.610350e+12,33287.33,33287.34,33238.66,33238.66,3.071353
84254,1.610350e+12,33237.06,33301.76,33227.19,33300.83,3.109293


In [17]:
df["time"] = list(map(lambda x: datetime.datetime.fromtimestamp(x/1000), df.time))
df = df.set_index(["time"], drop=True)
df = df[~df.index.duplicated(keep='first')]
len(df)

79373

In [25]:
df.to_csv("../data/testing.csv")